# MCP Client in Jupyter Notebook
This notebook demonstrates how to connect to an MCP math server and use LangChain tools.

In [1]:
import sys
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

async def run_mcp(chat_history=None):
    math_params = StdioServerParameters(
        command=sys.executable,
        args=["d:\\MAT496-Monsoon2025\\mod04_MCP\\servers\\math_server.py"],
    )
    fetch_params = StdioServerParameters(
        command=sys.executable,
        args=["-m", "mcp_server_fetch"],  # assumes installed via pip
    )
    sequentialthinking_params = StdioServerParameters(
        command="npx",
        args=["-y", "@modelcontextprotocol/server-sequential-thinking"],  # requires Node.js and npx
    )

    async with stdio_client(math_params, errlog=None) as (math_read, math_write), \
               stdio_client(fetch_params, errlog=None) as (fetch_read, fetch_write), \
               stdio_client(sequentialthinking_params, errlog=None) as (seq_read, seq_write):
        async with ClientSession(math_read, math_write) as math_session, \
                   ClientSession(fetch_read, fetch_write) as fetch_session, \
                   ClientSession(seq_read, seq_write) as seq_session:
            await math_session.initialize()
            await fetch_session.initialize()
            await seq_session.initialize()
            math_tools = await load_mcp_tools(math_session)
            fetch_tools = await load_mcp_tools(fetch_session)
            seq_tools = await load_mcp_tools(seq_session)
            tools = math_tools + fetch_tools + seq_tools
            agent = create_react_agent("groq:openai/gpt-oss-120b", tools)
            result = await agent.ainvoke(chat_history)
            return result

In [2]:
chat_history = {"messages": [{"role": "user", "content": "hi?"}]}

chat_history = await run_mcp(chat_history)
chat_history

{'messages': [HumanMessage(content='hi?', additional_kwargs={}, response_metadata={}, id='1b26801d-abbe-4197-b4e5-c7ea121b2595'),
  AIMessage(content='Hello! How can I help you today?', additional_kwargs={'reasoning_content': 'User just says "hi?" Probably greeting. Should respond friendly.'}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 1035, 'total_tokens': 1067, 'completion_time': 0.063677376, 'prompt_time': 0.037959891, 'queue_time': 0.054026699, 'total_time': 0.101637267}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ed9190d8b7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6cb28768-7a9e-480f-b6b0-16120a2bacdb-0', usage_metadata={'input_tokens': 1035, 'output_tokens': 32, 'total_tokens': 1067})]}

In [3]:
new_message = "Go the the site https://krajit.github.io/ and find Ajit's email address"

chat_history["messages"].append({"role": "user", "content": new_message})
chat_history = await run_mcp(chat_history)
chat_history

{'messages': [HumanMessage(content='hi?', additional_kwargs={}, response_metadata={}, id='1b26801d-abbe-4197-b4e5-c7ea121b2595'),
  AIMessage(content='Hello! How can I help you today?', additional_kwargs={'reasoning_content': 'User just says "hi?" Probably greeting. Should respond friendly.'}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 1035, 'total_tokens': 1067, 'completion_time': 0.063677376, 'prompt_time': 0.037959891, 'queue_time': 0.054026699, 'total_time': 0.101637267}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ed9190d8b7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6cb28768-7a9e-480f-b6b0-16120a2bacdb-0', usage_metadata={'input_tokens': 1035, 'output_tokens': 32, 'total_tokens': 1067}),
  HumanMessage(content="Go the the site https://krajit.github.io/ and find Ajit's email address", additional_kwargs={}, response_metadata={}, id='175e038a-f15e-44df-8903-3590dae8ce25'),
  AIMessage(conte